In [45]:
import torch
import torchvision
import torchvision.transforms as transforms
import torch.nn as nn
import torch.nn.functional as F
import numpy as np
from fastonn import SelfONN2d
from matplotlib import pyplot as plt



In [46]:
transform = transforms.Compose(
    [transforms.ToTensor(),
     transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])

batch_size = 4

trainset = torchvision.datasets.CIFAR10(root='./data', train=True,
                                        download=True, transform=transform)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=batch_size,
                                          shuffle=True, num_workers=2)

testset = torchvision.datasets.CIFAR10(root='./data', train=False,
                                       download=True, transform=transform)
testloader = torch.utils.data.DataLoader(testset, batch_size=batch_size,
                                         shuffle=False, num_workers=2)
print(type(testloader))

classes = ('plane', 'car', 'bird', 'cat',
           'deer', 'dog', 'frog', 'horse', 'ship', 'truck')

Files already downloaded and verified
Files already downloaded and verified
<class 'torch.utils.data.dataloader.DataLoader'>


In [47]:
class Net(nn.Module):
    def __init__(self):
        super().__init__()
        self.self1 = SelfONN2d(3, 6, 5, q=7)
        self.pool = nn.MaxPool2d(2, 2)
        self.self2 = SelfONN2d(6, 16, 5, q = 7)
        self.fc1 = nn.Linear(16 * 5 * 5, 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 10)

    def forward(self, x):
        x = self.pool(F.relu(self.self1(x)))
        x = self.pool(F.relu(self.self2(x)))
        x = torch.flatten(x, 1) # flatten all dimensions except batch
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x


net = Net()



In [48]:
import torch.optim as optim

criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(net.parameters(), lr=0.001, momentum=0.9)

In [49]:
epochs = 5
for epoch in range(epochs):  # loop over the dataset multiple times

    running_loss = 0.0
    for i, data in enumerate(trainloader, 0):
        # get the inputs; data is a list of [inputs, labels]
        inputs, labels = data

        # zero the parameter gradients
        optimizer.zero_grad()

        # forward + backward + optimize
        outputs = net(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        # print statistics
        running_loss += loss.item()
        if i % 2000 == 1999:    # print every 2000 mini-batches
            print(f'[{epoch + 1}, {i + 1:5d}] loss: {running_loss / 2000:.3f}')
            running_loss = 0.0

print('Finished Training')

[1,  2000] loss: 2.308
[1,  4000] loss: 2.261
[1,  6000] loss: 2.153
[1,  8000] loss: 2.112
[1, 10000] loss: 2.049
[1, 12000] loss: 1.992
[2,  2000] loss: 1.910
[2,  4000] loss: 1.861
[2,  6000] loss: 1.844
[2,  8000] loss: 1.950
[2, 10000] loss: 1.925
[2, 12000] loss: 1.871
[3,  2000] loss: 1.835
[3,  4000] loss: 1.809
[3,  6000] loss: 1.809
[3,  8000] loss: 1.766
[3, 10000] loss: 1.769
[3, 12000] loss: 1.790
[4,  2000] loss: 1.759
[4,  4000] loss: 1.755
[4,  6000] loss: 1.700
[4,  8000] loss: 1.729
[4, 10000] loss: 1.771
[4, 12000] loss: 1.747
[5,  2000] loss: 1.730
[5,  4000] loss: 1.764
[5,  6000] loss: 1.717
[5,  8000] loss: 1.735
[5, 10000] loss: 1.731
[5, 12000] loss: 1.822
Finished Training


In [50]:
correct = 0
total = 0
# since we're not training, we don't need to calculate the gradients for our outputs
with torch.no_grad():
    for data in testloader:
        images, labels = data
        # calculate outputs by running images through the network
        outputs = net(images)
        # the class with the highest energy is what we choose as prediction
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print(f'Accuracy of the network on the 10000 test images: {100 * correct // total} %')

Accuracy of the network on the 10000 test images: 32 %
